In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from datasets import Dataset
from transformers import TrainingArguments
from transformers import Trainer, DataCollatorForSeq2Seq
import json

# GPU가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # 출력: 'cuda'

In [ ]:
# 모델 및 토크나이저 설정
MODEL_NAME = "hyunwoongko/kobart"
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

In [ ]:
# 제주 JSON 데이터 로드 및 전처리

# 제주 JSON 파일 경로 설정
json_file = "jeju_data.json"  # JSON 파일 이름
with open(json_file, "r", encoding="utf-8-sig") as f:
    data = json.load(f)

# Dataset 객체로 변환
dataset = Dataset.from_list(data)

In [4]:
# 전처리 함수
def preprocess_function(examples):
    inputs = tokenizer(examples["source"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["target"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
# 데이터셋 전처리
processed_dataset = dataset.map(preprocess_function, batched=False)

# 학습/평가 데이터 분리
train_test_split = processed_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


In [ ]:
# 제주 모델 파인튜닝
training_args = TrainingArguments(
    output_dir="./kobart-finetuned-jeju",  # 모델 출력 경로
    evaluation_strategy="epoch",  # 평가 전략 (매 epoch마다 평가)
    save_strategy="epoch",  # 체크포인트 저장 전략 (매 epoch마다 저장)
    learning_rate=5e-5,  # 학습률
    per_device_train_batch_size=8,  # 학습 시 배치 크기
    per_device_eval_batch_size=8,  # 평가 시 배치 크기
    num_train_epochs=3,  # 총 학습 epoch 수
    weight_decay=0.01,  # weight decay
    save_total_limit=2,  # 저장할 체크포인트의 최대 개수
    logging_dir="./jeju-logs",  # 로그 파일 저장 디렉토리
    logging_steps=10000,  # 로그 작성 빈도 (step 단위)
    report_to="none",  # 외부 로깅 사용 안 함 (TensorBoard 등 제외)
    load_best_model_at_end=True,  # 학습 종료 시 최상의 모델 로드
    metric_for_best_model="eval_loss",  # 최상의 모델 선택 기준
    greater_is_better=False,  # 낮은 eval_loss일수록 좋은 모델
    fp16=True,
)

In [ ]:
# 데이터 Collator 정의 (torch 관련 제거)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,  # DataCollatorForSeq2Seq를 사용
)

# 5. 모델 학습
trainer.train()

# 6. 학습된 모델 저장
trainer.save_model("./kobart-finetuned-jeju")
tokenizer.save_pretrained("./kobart-finetuned-jeju")

In [ ]:
# 2. 경상 JSON 데이터 로드 및 전처리

# 경상 JSON 파일 경로 설정
json_file = "kyeongsang_data.json"  # JSON 파일 이름
with open(json_file, "r", encoding="utf-8-sig") as f:
    data = json.load(f)

# Dataset 객체로 변환
dataset = Dataset.from_list(data)

In [ ]:
# 전처리 함수
def preprocess_function(examples):
    inputs = tokenizer(examples["source"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["target"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
# 데이터셋 전처리
processed_dataset = dataset.map(preprocess_function, batched=False)

# 학습/평가 데이터 분리
train_test_split = processed_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


In [ ]:
#경상 모델 파인튜닝
training_args = TrainingArguments(
    output_dir="./kobart-finetuned-kyeongsang",  # 모델 출력 경로
    evaluation_strategy="epoch",  # 평가 전략 (매 epoch마다 평가)
    save_strategy="epoch",  # 체크포인트 저장 전략 (매 epoch마다 저장)
    learning_rate=5e-5,  # 학습률
    per_device_train_batch_size=8,  # 학습 시 배치 크기
    per_device_eval_batch_size=8,  # 평가 시 배치 크기
    num_train_epochs=3,  # 총 학습 epoch 수
    weight_decay=0.01,  # weight decay
    save_total_limit=2,  # 저장할 체크포인트의 최대 개수
    logging_dir="./kyeongsang-logs",  # 로그 파일 저장 디렉토리
    logging_steps=10000,  # 로그 작성 빈도 (step 단위)
    report_to="none",  # 외부 로깅 사용 안 함 (TensorBoard 등 제외)
    load_best_model_at_end=True,  # 학습 종료 시 최상의 모델 로드
    metric_for_best_model="eval_loss",  # 최상의 모델 선택 기준
    greater_is_better=False,  # 낮은 eval_loss일수록 좋은 모델
    fp16=True,
)

In [ ]:
# 데이터 Collator 정의 (torch 관련 제거)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 4. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,  # DataCollatorForSeq2Seq를 사용
)

# 5. 모델 학습
trainer.train()

# 6. 학습된 모델 저장
trainer.save_model("./kobart-finetuned-kyeongsang")
tokenizer.save_pretrained("./kobart-finetuned-kyeongsang")


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 저장된 모델 경로
model_path = "./kobart-finetuned-jeju"

# 모델과 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

# 테스트 입력 데이터
input_text = "서껑 허는 건 뭐 , 보리ᄊᆞᆯ에 ᄑᆞᆺ도 서끄곡 , 콩도 서끄곡 ."

# 입력 텍스트를 토크나이즈
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 모델 추론
output_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 생성된 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print("Input Text:", input_text)
print("Generated Text:", output_text)


In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 저장된 모델 경로
model_path = "./kobart-finetuned-kyeongsang"

# 모델과 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

# 테스트 입력 데이터
input_text = "그 의사 아 자기 들어봤어요 내가 들은 의사는 무엇이라 했냐면 잠들고 엄청 시간이 길지 않니 이게 완전히"

# 입력 텍스트를 토크나이즈
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 모델 추론
output_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 생성된 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print("Input Text:", input_text)
print("Generated Text:", output_text)

In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# 제주 방언 -> 표준어 모델 경로
jeju_model_path = "./kobart-finetuned-jeju3"
# 표준어 -> 경상도 방언 모델 경로
gyeongsang_model_path = "./kobart-finetuned-kyeongsang"

# 모델과 토크나이저 로드 (제주 방언 -> 표준어)
jeju_model = BartForConditionalGeneration.from_pretrained(jeju_model_path)
jeju_tokenizer = PreTrainedTokenizerFast.from_pretrained(jeju_model_path)

# 모델과 토크나이저 로드 (표준어 -> 경상도 방언)
gyeongsang_model = BartForConditionalGeneration.from_pretrained(gyeongsang_model_path)
gyeongsang_tokenizer = PreTrainedTokenizerFast.from_pretrained(gyeongsang_model_path)

# 테스트 입력 데이터 (제주 방언)
input_text_jeju = "서껑 허는 건 뭐 , 보리ᄊᆞᆯ에 ᄑᆞᆺ도 서끄곡 , 콩도 서끄곡 ."

# 입력 텍스트 출력
print("Input Text (Jeju Dialect):", input_text_jeju)

# 1단계: 제주 방언 -> 표준어 번역
input_ids_jeju = jeju_tokenizer.encode(input_text_jeju, return_tensors="pt")
output_ids_jeju = jeju_model.generate(input_ids_jeju, max_length=50, num_beams=4, early_stopping=True)
output_text_jeju = jeju_tokenizer.decode(output_ids_jeju[0], skip_special_tokens=True)

# 표준어 결과 출력
print("Standard Language:", output_text_jeju)

# 2단계: 표준어 -> 경상도 방언 번역
input_ids_standard = gyeongsang_tokenizer.encode(output_text_jeju, return_tensors="pt")
output_ids_gyeongsang = gyeongsang_model.generate(input_ids_standard, max_length=50, num_beams=4, early_stopping=True)
output_text_gyeongsang = gyeongsang_tokenizer.decode(output_ids_gyeongsang[0], skip_special_tokens=True)

# 경상도 방언 결과 출력
print("Gyeongsang Dialect:", output_text_gyeongsang)